# ANÁLISE DE SENTIMENTO EM REVIEWS DE TRÊS ESTRELAS: UMA PROPOSTA DE CLASSIFICAÇÃO MULTIRRÓTULO

## Importação das Bibliotecas

In [ ]:
#Importa a Biblioteca Pandas para Análise de Dados
import pandas as pd

#Importa Matplotlib e Seaborn para Plotagem e Visualização de Dados
import matplotlib.pyplot as plt
import seaborn as sns 

#Importa bibliotecas necessárias para Nuvem de Palavras
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import numpy as np

#Importa bibliotecas necessárias para Preparação dos Dados para os Modelos de Aprendizado de Máquina
import re
import nltk
import spacy
import spacy.cli
spacy.cli.download("pt_core_news_sm")
import pt_core_news_sm
spc_pt = pt_core_news_sm.load()
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

#Importa bibliotecas necessárias para Desenvolvimento dos Modelos de Aprendizado de Máquina
from sklearn.metrics import hamming_loss, accuracy_score
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.svm import SVC #suport vector machine
from sklearn.naive_bayes import MultinomialNB #naive bayes
from sklearn.tree import DecisionTreeClassifier #arvore de decisão
from sklearn.metrics import accuracy_score,classification_report #metricas

## Processamento/Tratamento dos Dados


#### Task 0: Carregar o dataset B2W-Reviews01


In [ ]:
#Carrega o dataset completo
data = pd.read_csv('https://raw.githubusercontent.com/americanas-tech/b2w-reviews01/main/B2W-Reviews01.csv')
#Printa as primeiras cinco linhas do dataset
data.head()

In [ ]:
#Printa informações gerais do Dataset
data.info()

#### Task 01: Remover Atributos Indesejados

In [ ]:
data = data.drop(columns=
                 ["submission_date", "product_id", "product_name", "product_brand", "site_category_lv2", "reviewer_state"])

#### Task 02: Filtrar Avaliações de Três Estrelas

In [ ]:
#Seleciona apenas as avaliações de nota 3
tres_estrelas = [3]
data= data[data.overall_rating.isin(tres_estrelas)]

#Printa informações gerais do Dataset mais uma vez
data.info()

#### Task 03: Concatenar Colunas Review_Title + Review_Text

In [ ]:
#Cria uma nova coluna Review
data["review"] = data['review_title'] + " " + data['review_text']

#Exclui as colunas Review_title e Review_text
data= data.drop(columns=["review_title", "review_text"])

#Printa as primeiras cinco linhas do dataset
data.head()

#### Task 04: Remover Avaliações Vazias

In [ ]:
#Confere a quantidade de campos vazios por atributo
data.isnull().sum()

#Remove as avaliações vazias
data=data.dropna(subset=['review'])

#### Task 05: Remover Avaliações Duplicadas

In [ ]:
#Confere a quantidade de avaliações duplicadas
data.duplicated('review').sum()

#Remove as avaliações duplicadas
data =data.drop_duplicates(subset=['review'], keep='first')

#### Task 06: Adicionar a coluna para Anotação Manual

In [ ]:
#Cria coluna vazia
data= data.assign(anotação='')

data.head()

#### Task 07: Salvar Dataset de Três Estrelas  

In [ ]:
#Salva o dataset de três estrelas limpo
data_limpo_três = data.to_csv("data_limpo_três.csv")

#### Task 08: Salvar Amostra para Anotação

In [ ]:
#Retira uma amostra aleatória simples de avaliações de três estrelas a partir do dataset limpo
amostra_tres_anotacao = data.sample(n=5000, random_state=1)

#Salva a amostra
amostra_tres_anotacao.to_csv('amostra_tres_anotacao.csv')

## Análise e Exploração dos Dados

#### Task 0: Carregar o dataset B2W-Reviews01-Anotado Manualmente

In [ ]:
#Carrega o dataset anotado
data_anotado = pd.read_csv('../Documents/TCC/b2w-reviews-01-anotado.csv')
data_anotado.index.name = 'index'

#Printa as primeiras cinco linhas do dataset
data_anotado.head()

In [ ]:
#Printa informações gerais do dataset anotado
data_anotado.info()

#### Task 01: Checar a distribuição das Classes de Sentimento

In [ ]:
#Checa a distribuição das classes de sentimento
data_anotado.anotacao.value_counts().to_frame()

In [ ]:
#Plota a distribuição das classes de sentimento em um gráfico de barras 
data_anotado.anotacao.value_counts().plot(kind="bar", color=["orange", "green","red","yellow"]);

#Adiciona informações ao gráfico
plt.title("Distribuição das Classes de Sentimento")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.xticks(rotation=0);

#### Task 02: Analisar o atributo Recomendação a um Amigo 

In [ ]:
#Quantifica o atributo Recommend to a Friend
data_anotado.recommend_to_a_friend.value_counts().to_frame()

In [ ]:
#Plota a distribuição de Recomendação a um Amigo em um gráfico de barras 
data_anotado.recommend_to_a_friend.value_counts().plot(kind="bar", color=["green", "red"]);

#Adiciona informações ao gráfico
plt.title("Distribuição - Recomendação a um Amigo")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Recomendação")
plt.xticks(rotation=0);

In [ ]:
#Cruza Recomendação a um Amigo com a distribuição de Sentimento
pd.crosstab(data_anotado.anotacao, data_anotado.recommend_to_a_friend)

In [ ]:
# Cria um plot para comparação entre Recomendação x Sentimento
pd.crosstab(data_anotado.anotacao, data_anotado.recommend_to_a_friend).plot(kind="bar", figsize=(10,6), color=["red", "green"])

#Adiciona informações ao gráfico
plt.title("Recomendação a um Amigo X Sentimento do Cliente")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.legend(["Não (No)", "Sim (Yes)"])
plt.xticks(rotation=0);

#### Task 03: Analisar o atributo Gênero

In [ ]:
#Quantifica o atributo Reviewer Gender
data_anotado.reviewer_gender.value_counts().to_frame()

In [ ]:
#Cruza Gênero com a distribuição de Sentimento
pd.crosstab(data_anotado.anotacao, data_anotado.reviewer_gender)

In [ ]:
# Cria um plot para comparação entre Gênero x Sentimento
pd.crosstab(data_anotado.anotacao, data_anotado.reviewer_gender).plot(kind="bar", figsize=(10,6), color=["pink", "blue"])

#Adiciona informações ao gráfico
plt.title("Gênero X Sentimento dos Clientes")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.legend(["Mulher (Female)","Homem (Male)"])
plt.xticks(rotation=0); 

#### Task 04: Analisar o atributo Idade

In [ ]:
#Checa o ano de nascimento do Cliente mais novo
data_anotado.reviewer_birth_year.max()

In [ ]:
#Checa o ano de nascimento do Cliente mais velho
data_anotado.reviewer_birth_year.min()

In [ ]:
#Agrupa os Clientes da Geração Z - Faixa Etária escolhida para Exploração dos Dados
GenZ =data_anotado.loc[data_anotado["reviewer_birth_year"].between(1995, 2010)]

In [ ]:
#Checa quantidade de reviews pertencentes à Geração Z
len(GenZ)

In [ ]:
#Checa a distribuição das classes de Sentimento da Geração Z por ano de nascimento
pd.crosstab(GenZ.anotacao, GenZ.reviewer_birth_year)

In [ ]:
#Plota a distribuição das classes de Sentimento da GenZ em um gráfico de barras
GenZ.anotacao.value_counts().plot(kind="bar",color=['orange','green','yellow','red']);

#Adiciona informações ao gráfico
plt.title("Distribuição das Classes de Sentimento GenZ - Geral")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.xticks(rotation=0);

In [ ]:
#Checa avaliações neutras GenZ
neutras_GenZ = GenZ[GenZ['anotacao']=='neutro']
neutras_GenZ.review.to_frame().head()

#### Task 05: Analisar o atributo Categoria de Produto

In [ ]:
#Quantifica o atributo Site Category em relação à GenZ
GenZ.site_category_lv1.value_counts().to_frame()

In [ ]:
#Plota a distribuição das classes de Sentimento da categoria mais popular - Gen Z 
top_dpt_genz = GenZ[GenZ['site_category_lv1']=='Eletrodomésticos']
top_dpt_genz.anotacao.value_counts().plot(kind="bar", color=['red','yellow','orange']);

#Adiciona informações ao gráfico
plt.title("Distribuição das Classes de Sentimento GenZ - Eletrodomésticos")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.xticks(rotation=0);

In [ ]:
#Plota a distribuição das classes de Sentimento da segunda categoria mais popular - Gen Z 
top2_dpt_genz = GenZ[GenZ['site_category_lv1']=='Móveis']
top2_dpt_genz.anotacao.value_counts().plot(kind="bar", color=['orange','green','yellow','red']);

#Adiciona informações ao gráfico
plt.title("Distribuição das Classes de Sentimento Gen Z - Móveis")
plt.ylabel("Quantidade")
plt.xlabel("Classes de Sentimento")
plt.xticks(rotation=0);

#### Task 06: Construir Nuvens de Palavras 

In [ ]:
#Constrói uma nuvem de palavra considerando o dataset inteiro

#Seleciona os textos da coluna Review e agrupa em uma lista
textos_reviews = data_anotado['review']
lista_geral = " ".join(textos_reviews)

#Importa stopwords em português
stopwords = open('stopwords.txt').read()

#Transforma as stopwords em um lista
lista_stopwords_wc = stopwords.split(' \n')

#Inicializa uma Nuvem de Palavra
wordcloud_geral = WordCloud(stopwords=lista_stopwords_wc,
                      background_color = 'white', #cor de fundo
                      width = 1000, #largura
                      height = 500) #altura
               
#Gera uma Nuvem de Palavra
wordcloud_geral.generate(lista_geral)
plt.figure(figsize = (15, 7)) #tamanho do gráfico
plt.imshow(wordcloud_geral, interpolation = 'bilinear') #plotagem da nuvem de palavras
plt.axis('off') #remove as bordas
plt.show() #mostra a nuvem de palavra

In [ ]:
#Constrói uma nuvem de palavra considerando as reviews positivas

#Seleciona apenas as reviews positivas e agrupa em uma lista
nuvem_positivas = data_anotado[data_anotado['anotacao']=='positivo']
textos_positivas = nuvem_positivas['review']
lista_positivas = " ".join(textos_positivas)

#Inicializa uma Nuvem de Palavra
wordcloud_positivas = WordCloud(stopwords=lista_stopwords_wc,
                      background_color = 'white', #cor de fundo
                      width = 1000, #largura
                      height = 500) #altura
               
#Gera uma Nuvem de Palavra
wordcloud_positivas.generate(lista_positivas)
plt.figure(figsize = (15, 7)) #tamanho do gráfico
plt.imshow(wordcloud_positivas, interpolation = 'bilinear') #plotagem da nuvem de palavras
plt.axis('off') #remove as bordas
plt.show() #mostra a nuvem de palavra

In [ ]:
#Constrói uma nuvem de palavra considerando as reviews negativas

#Seleciona apenas as reviews negativas e agrupa em uma lista
nuvem_negativas = data_anotado[data_anotado['anotacao']=='negativo']
textos_negativas = nuvem_negativas['review']
lista_negativas = " ".join(textos_negativas)

#Inicializa uma Nuvem de Palavra
wordcloud_negativas = WordCloud(stopwords=lista_stopwords_wc,
                      background_color = 'white', #cor de fundo
                      width = 1000, #largura
                      height = 500) #altura
               
#Gera uma Nuvem de Palavra
wordcloud_negativas.generate(lista_negativas)
plt.figure(figsize = (15, 7)) #tamanho do gráfico
plt.imshow(wordcloud_negativas, interpolation = 'bilinear') #plotagem da nuvem de palavras
plt.axis('off') #remove as bordas
plt.show() #mostra a nuvem de palavra

## Preparação dos Dados para os Modelos de Aprendizado de Máquina

#### Task 01: Eliminar Reviews Neutras

In [ ]:
data_anotado = data_anotado[data_anotado['anotacao'] != 'neutro']
data_anotado.reset_index(drop=True, inplace=True)
data_anotado.index.name = 'index'

In [ ]:
data_anotado.anotacao.value_counts()

#### Task 02: Pré-Processar os textos da coluna Reviews

In [ ]:
#Pré-processamento dos textos da coluna Reviews 
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_pt = stopwords.words("portuguese")
stopwords_pt = [i for i in stopwords_pt if i not in ["não", "nenhum", "nada", "jamais", "nunca", "nem"]]

def limpa_texto(texto):
    texto = texto.lower() #Transforma texto em minúsculo
    texto = re.sub(r"[\W\d_]+", " ", texto) #Filtra apenas letras
    texto = [pal for pal in texto.split() if pal not in stopwords_pt] #Remove stopwords
    spc_texto = spc_pt(" ".join(texto))
    tokens = [word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in spc_texto] #Lemmatiza

    return " ".join(tokens)

#Aplica função na coluna Review
data_anotado['review'] = data_anotado['review'].apply(limpa_texto)

#### Task 03: Transformar classes de sentimento em colunas

In [ ]:
#Lista as classes de sentimento existentes no dataset
SENTIMENTOS = ['positivo','negativo']
 
#Transforma as classes em coluna com valores preenchidos 
for sentimento in SENTIMENTOS[::-1]:
    data_anotado.insert(1,sentimento,data_anotado.anotacao.apply(lambda x: 1 if sentimento in x else 0))

#### Task 04: Dividir os dados em Treino e Teste

In [ ]:
#Divide o Dataset em Treino e Teste
X = data_anotado["review"]
y = data_anotado[['positivo', 'negativo']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Task 05: Vetorizar os textos da coluna Reviews

In [ ]:
#Vetoriza as Reviews com TFIDF
vectorizer_tfidf = TfidfVectorizer(max_features=5000)
vectorizer_tfidf.fit(X)
X_train_tfidf = vectorizer_tfidf.transform(X_train)
X_test_tfidf = vectorizer_tfidf.transform(X_test)

In [ ]:
#Vetoriza as Reviews com BOW
vectorizer_bow = CountVectorizer(binary=True)
vectorizer_bow.fit(X)
X_train_bow = vectorizer_bow.transform(X_train)
X_test_bow = vectorizer_bow.transform(X_test)

## Aplicação de Modelos de Aprendizado de Máquina 

#### Task 01: Testar o Método BR com Support Vector Machine (SVM)

In [ ]:
classifier_svm = BinaryRelevance(
    classifier = SVC())

In [ ]:
# Treina com TFIDF
classifier_svm.fit(X_train_tfidf, y_train)

# Faz a predição com TFIDF
predictions_tfidf_svm = classifier_svm.predict(X_test_tfidf)

# Treina com BOW
classifier_svm.fit(X_train_bow, y_train)

# Faz a predição com BOW
predictions_bow_svm = classifier_svm.predict(X_test_bow)

In [ ]:
# Acurácia SVM com TFIDF
accuracy_score(y_test,predictions_tfidf_svm)

In [ ]:
# Hamming Loss SVM com TFDIF
hamming_loss(y_test, predictions_tfidf_svm)

In [ ]:
# Acurácia SVM com BOW
accuracy_score(y_test,predictions_bow_svm)

In [ ]:
# Hamming Loss SVM com BOW
hamming_loss(y_test, predictions_bow_svm)

#### Task 02: Testar o Método BR com Naive Bayes (NB)

In [ ]:
classifier_nb = BinaryRelevance(
    classifier = MultinomialNB())

In [ ]:
# Treina com TFIDF
classifier_nb.fit(X_train_tfidf, y_train)

# Faz a predição com TFIDF
predictions_tfidf_nb = classifier_nb.predict(X_test_tfidf)

# Treina com BOW
classifier_nb.fit(X_train_bow, y_train)

# Faz a predição com BOW
predictions_bow_nb = classifier_nb.predict(X_test_bow)

In [ ]:
# Acurácia NB com TFIDF
accuracy_score(y_test,predictions_tfidf_nb)

In [ ]:
# Hamming Loss NB com TFDIF
hamming_loss(y_test, predictions_tfidf_nb)

In [ ]:
# Acurácia NB com BOW
accuracy_score(y_test,predictions_bow_nb)

In [ ]:
# Hamming Loss NB com BOW
hamming_loss(y_test, predictions_bow_nb)

#### Task 03: Testar o Método BR com Árvore de Decisão (AD)

In [ ]:
classifier_ad = BinaryRelevance(
    classifier = DecisionTreeClassifier(criterion='entropy'))

In [ ]:
# Treina com TFIDF
classifier_ad.fit(X_train_tfidf, y_train)

# Faz a predição com TFIDF
predictions_tfidf_ad = classifier_ad.predict(X_test_tfidf)

# Treina com BOW
classifier_ad.fit(X_train_bow, y_train)

# Faz a predição com BOW
predictions_bow_ad = classifier_ad.predict(X_test_bow)

In [ ]:
# Acurácia AD com TFIDF
accuracy_score(y_test,predictions_tfidf_ad)

In [ ]:
# Hamming Loss AD com TFDIF
hamming_loss(y_test, predictions_tfidf_ad)

In [ ]:
# Acurácia AD com BOW
accuracy_score(y_test,predictions_bow_ad)

In [ ]:
# Hamming Loss NB com BOW
hamming_loss(y_test, predictions_bow_ad)

### MÉTRICAS - Parte Visual

In [ ]:
# Gráfico Barras Agrupadas - Acurácia
acuraciabow = [0.80, 0.77, 0.74]
acuraciatfidf = [0.78, 0.69, 0.73]
 
# Definindo a largura das barras
barWidth = 0.25

# Aumentando o gráfico
plt.figure(figsize=(12,7))

# Definindo a posição das barras
r1 = np.arange(len(acuraciabow))
r2 = [x + barWidth for x in r1]

 
# Criando as barras
plt.bar(r1, acuraciabow, color='#24376D', width=barWidth, label='Accuracy - BoW')
plt.bar(r2, acuraciatfidf, color='#6D5A24', width=barWidth, label='Accuracy - TF-IDF')
 
# Adiciando legendas as barras
plt.xlabel('Modelos')
plt.xticks([r + barWidth for r in range(len(acuraciabow))], ['SVM', 'NB', 'AD'])
plt.ylabel('Valores')
 
# Criando a legenda e exibindo o gráfico
plt.legend()
plt.show()

In [ ]:
# Gráfico Barras Agrupadas - HammingLoss
hamminglossbow = [0.09, 0.11, 0.13]
hamminglosstfidf = [0.10, 0.15, 0.13]
 
# Definindo a largura das barras
barWidth = 0.25

# Aumentando o gráfico
plt.figure(figsize=(10,5))

# Definindo a posição das barras
r1 = np.arange(len(hamminglossbow))
r2 = [x + barWidth for x in r1]

 
# Criando as barras
plt.bar(r1, hamminglossbow, color='#5A246D', width=barWidth, label='Hamming Loss - BoW')
plt.bar(r2, hamminglosstfidf, color='#6495ED', width=barWidth, label='Hamming Loss - TF-IDF')
 
# Adiciando legendas as barras
plt.xlabel('Modelos')
plt.xticks([r + barWidth for r in range(len(hamminglossbow))], ['SVM', 'NB', 'AD'])
plt.ylabel('Valores')
 
# Criando a legenda e exibindo o gráfico
plt.legend()
plt.show()